# <a id='toc1_'></a>[__Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком__](#toc0_)

**Содержание**<a id='toc0_'></a>    
- [__Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком__](#toc1_)    
  - [__Описание исследования__](#toc1_1_)    
  - [__Постановка задачи__](#toc1_2_)    
  - [__Импорты модулей__](#toc1_3_)    
  - [__Обзор данных__](#toc1_4_)    
  - [__Часть 1. Критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках__](#toc1_5_)    
  - [__Часть 2. Поправка Бонферрони, поправка Холма__](#toc1_6_)    
  - [__Часть 3. Поправка Бенджамини-Хохберга__](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

***
## <a id='toc1_1_'></a>[__Описание исследования__](#toc0_)

Данные взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы точнее диагностировать рак груди на ранних стадиях.

В исследовании участвовали $24$ человека, у которых нет рака груди (`normal`), $25$ человек, у которых заболевание было диагностировано на ранней стадии (`early neoplasia`), и $23$ человека с сильно выраженными симптомами (`cancer`).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчета количества соответствующей каждому гену РНК.

В данных представлена количественная мера активности каждого из $15748$ генов у каждого из $72$ человек, принимавших участие в исследовании. Диагноз испытуемого в столбце `Diagnosis`.

***
## <a id='toc1_2_'></a>[__Постановка задачи__](#toc0_)

1. Определить гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

2. Оценить практическую значимость результатов — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого используется метрика __fold change__ (_кратность изменения_):

$$\large F_c \left(C,T\right) = \begin{cases}\frac{T}{C}, & T > C \\ 
-\frac{C}{T}, & T < C\end{cases},$$

где $C, T$ — средние значения экспрессии гена в control и treatment группах соответственно. Фактически fold change показывает, во сколько раз отличаются средние двух выборок.

***
## <a id='toc1_3_'></a>[__Импорты модулей__](#toc0_)

In [ ]:
# стандартная библиотека
import collections
import dataclasses

In [ ]:
# сторонние библиотеки
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests
from statsmodels.stats.descriptivestats import describe
from tqdm.notebook import tqdm
import black
import numpy as np
import pandas as pd

***
## <a id='toc1_4_'></a>[__Обзор данных__](#toc0_)

In [3]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.sample(10)  # случайная выборка из данных объема 10

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
15,STT5611_Breast_017_normal,normal,2.971205,4.392145,14.134202,14.134202,21.356879,9.619310,7.300364,61.636088,...,5.689424,0.990402,0.990402,0.990402,0.990402,0.990402,15.959505,0.990402,0.990402,0.990402
7,STT5466_Breast_007_normal,normal,3.153900,1.647070,4.941211,11.529492,13.813151,8.235352,1.647070,44.226216,...,10.546396,1.647070,1.647070,1.647070,1.647070,1.647070,21.071346,1.647070,1.647070,1.647070
47,STT5830_Breast_024_EN,early neoplasia,1.122588,12.750152,30.695485,24.620283,30.309883,9.050596,10.747964,64.770521,...,4.628550,1.122588,1.122588,1.122588,1.122588,1.122588,15.702866,1.122588,1.122588,1.122588
36,STT5571_Breast_013_EN,early neoplasia,0.949252,7.960901,20.933827,13.368401,25.955872,13.546952,9.602630,60.183389,...,6.461426,1.817680,0.949252,0.949252,0.949252,0.949252,19.873935,0.949252,0.949252,0.949252
27,STT5445_Breast_005_EN,early neoplasia,2.131757,8.789458,12.731187,6.395270,19.185811,14.922297,4.082003,52.028259,...,9.453726,2.131757,2.131757,2.131757,2.131757,2.131757,23.189490,2.131757,2.131757,2.131757
43,STT5667_Breast_020_EN,early neoplasia,1.285190,7.382856,14.747056,13.661760,22.148568,9.473285,7.078276,58.364242,...,6.759987,9.927349,9.927349,10.778244,6.425951,10.361535,10.146765,9.702974,13.499597,7.078276
14,STT5576_Breast_015_normal,normal,2.561871,2.916818,10.654518,13.831291,26.582626,8.969936,7.934604,71.859351,...,3.787053,0.853957,0.853957,0.853957,0.853957,0.853957,17.824328,0.853957,0.853957,0.853957
58,STT5575_Breast_014_IDC,cancer,3.331313,7.558592,25.346209,17.142989,30.953137,13.123224,10.317712,70.472701,...,4.578452,1.110438,1.110438,1.110438,1.110438,1.110438,11.804114,1.110438,1.110438,1.110438
54,STT5477_Breast_009_IDC,cancer,1.126738,4.996760,13.442388,11.691325,21.169493,8.703401,6.729059,58.340683,...,7.214646,1.126738,1.126738,1.126738,1.126738,1.126738,8.098943,1.126738,1.126738,1.126738
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576


In [4]:
# описательные статистики
describe(data)

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
nobs,72.000000,72.000000,7.200000e+01,72.000000,72.000000,72.000000,72.000000,7.200000e+01,7.200000e+01,72.000000,...,72.000000,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01
missing,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
mean,2.463922,7.100958,1.905615e+01,15.618688,23.537120,11.303466,8.921396,6.327015e+01,5.390632e+01,70.678573,...,5.563444,2.192029e+00,1.967424e+00,2.181360e+00,1.729308e+00,1.980733e+00,1.683408e+01,1.825827e+00,2.287380e+00,1.756827e+00
std_err,0.166605,0.490815,1.094572e+00,0.549675,0.527418,0.477901,0.385457,1.625419e+00,5.378195e+00,2.513321,...,0.281639,2.892324e-01,2.412564e-01,2.996544e-01,1.776192e-01,2.628265e-01,8.399017e-01,2.242372e-01,3.754231e-01,1.912902e-01
upper_ci,2.790461,8.062937,2.120147e+01,16.696031,24.570842,12.240134,9.676878,6.645591e+01,6.444739e+01,75.604592,...,6.115447,2.758914e+00,2.440277e+00,2.768671e+00,2.077435e+00,2.495864e+00,1.848025e+01,2.265324e+00,3.023195e+00,2.131749e+00
lower_ci,2.137383,6.138978,1.691083e+01,14.541345,22.503399,10.366798,8.165913,6.008438e+01,4.336526e+01,65.752554,...,5.011441,1.625144e+00,1.494570e+00,1.594048e+00,1.381181e+00,1.465603e+00,1.518790e+01,1.386330e+00,1.551564e+00,1.381905e+00
std,1.413687,4.164703,9.287753e+00,4.664146,4.475294,4.055122,3.270713,1.379214e+01,4.563550e+01,21.326237,...,2.389789,2.454218e+00,2.047129e+00,2.542652e+00,1.507149e+00,2.230157e+00,7.126802e+00,1.902716e+00,3.185571e+00,1.623151e+00
iqr,2.060426,5.023995,8.460379e+00,5.121083,5.320285,4.855727,4.925844,1.678708e+01,1.628607e+01,25.225107,...,3.295067,7.729704e-01,6.359055e-01,6.666210e-01,6.220275e-01,6.079621e-01,1.066275e+01,6.040081e-01,6.359055e-01,6.079621e-01
iqr_normal,1.527396,3.724293,6.271688e+00,3.796264,3.943933,3.599556,3.651534,1.244428e+01,1.207288e+01,18.699400,...,2.442637,5.730038e-01,4.713975e-01,4.941669e-01,4.611097e-01,4.506830e-01,7.904310e+00,4.477519e-01,4.713975e-01,4.506830e-01
mad,1.143538,3.302321,6.367588e+00,3.620146,3.332341,3.057918,2.712135,1.000034e+01,2.614637e+01,16.507661,...,1.946500,1.487371e+00,1.192385e+00,1.517612e+00,8.502117e-01,1.271065e+00,5.528517e+00,1.025972e+00,1.734240e+00,9.063246e-01


In [5]:
# пропущенных значений нет
np.count_nonzero(data.isna())

0

***
## <a id='toc1_5_'></a>[__Часть 1. Критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках__](#toc0_)

1. Применить t-критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках для каждого гена дважды:
    * для групп `normal` (control) и `early neoplasia` (treatment);
    * для групп `early neoplasia` (control) и `cancer` (treatment).

2. Найти число отклонений нулевой гипотезы, статистически значимых на уровне $0.05$.

In [6]:
# пары диагнозов для извлечения выборок для тестирования
diagnoses = [('normal', 'early neoplasia'), ('early neoplasia', 'cancer')]

In [7]:
# базовый уровень статистической значимости
ALPHA = 0.05

In [8]:
@dataclasses.dataclass
class ControlTreatmentCase:
    """
    Кейс тестирования.
    Хранилище результатов тестирования выборок,
    соответствующих паре диагнозов diagnoses.
    """
    # пара диагнозов данного кейса тестирования
    diagnoses: tuple[str, str]
    
    # счетчики числа отклонения нулевой гипотезы:
    no_adjust: int = 0  # без корректировки уровня значимости
    bonf_holm: int = 0  # с поправками Бонферрони и Холма
    benj_hoch: int = 0  # с поправками Бонферрони и Бенджамини-Хохберга
    pvalues: list[float] = dataclasses.field(default_factory=list)
    
    def __repr__(self) -> str:
        attrs = {**self.__dict__}
        attrs['pvals_min'] = min(attrs['pvalues'])
        attrs['pvals_max'] = max(attrs['pvalues'])
        del attrs['pvalues']
        return black.format_str(str(attrs), mode=black.Mode())

In [9]:
# инициализированные объекты хранилищ результатов (кейсы тестирования)
normal_early, early_cancer = map(ControlTreatmentCase, diagnoses)
cases = [normal_early, early_cancer]

In [10]:
def get_subsets(case: ControlTreatmentCase) -> list[pd.DataFrame]:
    """
    Разбиение исходного набора данных по диагнозам данного case. 
    """
    return [data.query(f'Diagnosis == "{diag}"') for diag in case.diagnoses]

Перед проведением двухвыборочного t-теста на равенство средних независимых выборок рекомендуется провести тест Фишера на равенство дисперсий:

* [__F-test in Python__](https://stackoverflow.com/questions/21494141/how-do-i-do-a-f-test-in-python)
* [__scipy.stats.f__](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f.html#scipy-stats-f)

In [11]:
# кастомный тип данных
array_like = list[float] | tuple[float] | np.ndarray | pd.Series

In [12]:
# результат теста Фишера именованным кортежем
fields = ['equal_var', 'pvalue']
FisherTestResult = collections.namedtuple('FisherTestResult', fields)

In [13]:
def variance_equivalence_fisher_test(
    a: array_like, b: array_like, alpha: float = ALPHA
) -> FisherTestResult:
    """
    F-тест Фишера на равенство дисперсий.
    """
    # статистика теста Фишера
    F = np.var(a, ddof=1) / np.var(b, ddof=1)
    
    # n - numerator/числитель, d - denominator/знаменатель 
    dfn, dfd = np.size(a) - 1, np.size(b) - 1
    
    # sf - survival function (1 - cdf)
    pvalue = stats.f.sf(F, dfn, dfd)
    
    equal_var = pvalue >= alpha
    return FisherTestResult(equal_var, pvalue)

[__scipy.stats.ttest_ind__](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy-stats-ttest-ind)
> Calculate the T-test for the means of two independent samples of scores.<br>
This is a test for the null hypothesis that 2 independent samples have identical average (expected) values.

In [14]:
for case in cases:
    subsets = get_subsets(case)
        
    # тестирование одноименных генов-столбцов из разных subset 
    for gene in tqdm(data.columns[2:], desc=str(case.diagnoses)):
        samples = [subset[gene] for subset in subsets]
        
        # тест Фишера на равенство дисперсий
        res = variance_equivalence_fisher_test(*samples)
        
        # двухвыборочный t-тест на равенство средних независимых выборок
        # alternative='two-sided' по умолчанию
        pvalue = stats.ttest_ind(*samples, equal_var=res.equal_var).pvalue
        
        # сбор pvalue, обновление счетчика числа отклонений нулевой гипотезы
        case.pvalues.append(pvalue)
        case.no_adjust += (pvalue < ALPHA)

('normal', 'early neoplasia'):   0%|          | 0/15748 [00:00<?, ?it/s]

('early neoplasia', 'cancer'):   0%|          | 0/15748 [00:00<?, ?it/s]

In [15]:
# обновленные тест-кейсы
cases

[{
     "diagnoses": ("normal", "early neoplasia"),
     "no_adjust": 1572,
     "bonf_holm": 0,
     "benj_hoch": 0,
     "pvals_min": 5.324929009240687e-08,
     "pvals_max": 0.9999197791597287,
 },
 {
     "diagnoses": ("early neoplasia", "cancer"),
     "no_adjust": 3541,
     "bonf_holm": 0,
     "benj_hoch": 0,
     "pvals_min": 7.441990111722422e-11,
     "pvals_max": 0.999982055535946,
 }]

***
## <a id='toc1_6_'></a>[__Часть 2. Поправка Бонферрони, поправка Холма__](#toc0_)

1. Применить __поправку Холма__ для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Отметим, что поскольку поправка для каждого из двух наборов pvalue производится отдельно, проблема, связанная с множественной проверкой остается.

Для того, чтобы ее устранить, воспользуемся __поправкой Бонферрони__: установим уровень значимости $0.025 = 0.05 / 2$ вместо $0.05$ для дальнейшего уточнения значений pvalue c помощью метода Холма.

2. Найти число отклонений нулевой гипотезы (количество значимых отличий) в каждой группе после коррекции Холма-Бонферрони с учетом практической значимости: с учетом ограничения на метрику fold change $|F_c| > 1.5$.

Особо отметим:
* применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только к тем, которые меньше базового уровня значимости `ALPHA`.

* при использовании поправки на уровне значимости $0.025$ меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия: при отборе значимых отличий значения уровня значимости нужно сравнивать с порогом $0.025$, а не $0.05$.

In [16]:
def fold_change(control: array_like, treatment: array_like) -> float:
    """
    Метрика кратности изменения. Отношение средних значений экспрессии гена.
    """
    cmean, tmean = map(np.mean, (control, treatment))
    return (-cmean/tmean, tmean/cmean)[int(tmean > cmean)]

# int: использование np.bool_ в качестве индекса deprecated 

In [17]:
# нижний невключаемый порог кратности изменения
FOLD_CHANGE_THRESHOLD = 1.5

In [18]:
# поправка Бонферрони для уровня значимости, 0.05 / 2 = 0.025
alpha = ALPHA / len(cases)

In [19]:
# словарь соответствия поправок атрибутам кейсов тестирования
adjustments = {'holm': 'bonf_holm', 'fdr_bh': 'benj_hoch'}

[__Multiple Tests and Multiple Comparison Procedures__](https://www.statsmodels.org/dev/stats.html#multiple-tests-and-multiple-comparison-procedures)

In [20]:
def multitest(cases: list[ControlTreatmentCase], method: str
              ) -> list[ControlTreatmentCase]:
    """
    Множественное тестирование гипотез в кейсах cases с поправкой method
    и проверкой практической значимости статистических результатов метрикой 
    fold change.
    """
    for case in cases:
        subsets = get_subsets(case)
        
        # True для H0, которые могут быть отклонены на данном ур. значимости
        rejects = multipletests(case.pvalues, alpha=alpha, method=method)[0]
        
        # индексы отклонений с поправкой на столбцы Patient_id и Diagnosis
        indices = np.nonzero(rejects)[0] + 2
        
        # проверка практической значимости стат. результатов с пом. fold change
        for index in tqdm(indices, desc=str(case.diagnoses)):
            control, treatment = [subset.iloc[:,index] for subset in subsets]
            fold_change_ratio = fold_change(control, treatment)
            effective = np.abs(fold_change_ratio) > FOLD_CHANGE_THRESHOLD
            case.__dict__[adjustments[method]] += effective
            
    return cases

In [21]:
multitest(cases, method='holm')

('normal', 'early neoplasia'):   0%|          | 0/2 [00:00<?, ?it/s]

('early neoplasia', 'cancer'):   0%|          | 0/84 [00:00<?, ?it/s]

[{
     "diagnoses": ("normal", "early neoplasia"),
     "no_adjust": 1572,
     "bonf_holm": 2,
     "benj_hoch": 0,
     "pvals_min": 5.324929009240687e-08,
     "pvals_max": 0.9999197791597287,
 },
 {
     "diagnoses": ("early neoplasia", "cancer"),
     "no_adjust": 3541,
     "bonf_holm": 81,
     "benj_hoch": 0,
     "pvals_min": 7.441990111722422e-11,
     "pvals_max": 0.999982055535946,
 }]

***
## <a id='toc1_7_'></a>[__Часть 3. Поправка Бенджамини-Хохберга__](#toc0_)

Часть аналогичная Части 2, но с поправкой Бенджамини-Хохберга. 

In [22]:
multitest(cases, method='fdr_bh')

('normal', 'early neoplasia'):   0%|          | 0/4 [00:00<?, ?it/s]

('early neoplasia', 'cancer'):   0%|          | 0/912 [00:00<?, ?it/s]

[{
     "diagnoses": ("normal", "early neoplasia"),
     "no_adjust": 1572,
     "bonf_holm": 2,
     "benj_hoch": 4,
     "pvals_min": 5.324929009240687e-08,
     "pvals_max": 0.9999197791597287,
 },
 {
     "diagnoses": ("early neoplasia", "cancer"),
     "no_adjust": 3541,
     "bonf_holm": 81,
     "benj_hoch": 561,
     "pvals_min": 7.441990111722422e-11,
     "pvals_max": 0.999982055535946,
 }]

***